<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/lightcat_allcal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression


# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [5]:
df_train.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,MaritalStatus,OwnCar,NumberOfChildren,Agebin,Incomebin
0,0,50.0,Self Enquiry,2,15.0,Large Business,MALE,1.0,4.0,BASIC,...,1,4,Executive,253905.0,1.0,未婚,なし,0.0,41-50,<300000
1,1,56.0,Company Invited,1,14.0,Salaried,MALE,1.0,4.0,STANDARD,...,1,4,Senior Manager,404475.0,0.0,離婚,あり,0.0,51-65,<450000
2,2,NaN,Self Enquiry,1,10.0,Large Business,FEMALE,1.0,3.0,BASIC,...,0,4,Executive,278145.0,1.0,結婚,なし,0.0,NaN,<300000
3,3,37.0,Self Enquiry,2,18.0,Small Business,FEMALE,1.0,3.0,STANDARD,...,0,5,Senior Manager,326805.0,0.0,離婚,あり,0.0,31-40,<350000
4,4,48.0,Company Invited,3,17.0,Small Business,FEMALE,1.0,3.0,BASIC,...,0,4,Executive,258435.0,1.0,独身,あり,0.0,41-50,<300000


In [6]:
print(df_train.shape)
print(df_train)

(3489, 22)
        id   Age    TypeofContact  CityTier  DurationOfPitch      Occupation  \
0        0  50.0     Self Enquiry         2             15.0  Large Business   
1        1  56.0  Company Invited         1             14.0        Salaried   
2        2   NaN     Self Enquiry         1             10.0  Large Business   
3        3  37.0     Self Enquiry         2             18.0  Small Business   
4        4  48.0  Company Invited         3             17.0  Small Business   
...    ...   ...              ...       ...              ...             ...   
3484  3484  40.0     Self Enquiry         2             26.0        Salaried   
3485  3485  40.0     Self Enquiry         1              9.0  Large Business   
3486  3486  31.0     Self Enquiry         1             14.0  Small Business   
3487  3487  56.0  Company Invited         2             15.0        Salaried   
3488  3488  42.0     Self Enquiry         1              9.0  Small Business   

      Gender  NumberOfPerson

In [7]:
rows = []
for col in df_train.columns:
    rows.append([col, df_train[col].dtype, df_train[col].isnull().sum(), len(df_train[col].unique())])
pd.DataFrame(rows, columns=["列名", "列の型", "NaNである行の数", "値の種類"])

,列名,列の型,NaNである行の数,値の種類
0,id,int64,0,3489
1,Age,float64,100,45
2,TypeofContact,object,6,3
3,CityTier,int64,0,3
4,DurationOfPitch,float64,121,34
5,Occupation,object,0,3
6,Gender,object,0,2
7,NumberOfPersonVisiting,float64,0,4
8,NumberOfFollowups,float64,33,7
9,ProductPitched,object,0,5


In [8]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,3489.0,1744.000000,1007.331872,0.0,872.0,1744.0,2616.0,3488.0
Age,3389.0,39.022721,9.779403,18.0,31.0,38.0,47.0,61.0
CityTier,3489.0,1.697048,0.671500,1.0,1.0,2.0,2.0,3.0
DurationOfPitch,3368.0,14.344715,6.954363,4.0,9.0,13.0,16.0,36.0
NumberOfPersonVisiting,3489.0,2.394669,0.726612,1.0,2.0,2.0,3.0,4.0
NumberOfFollowups,3456.0,3.668981,0.889123,1.0,3.0,4.0,4.0,6.0
PreferredPropertyStar,3489.0,3.441101,0.599633,3.0,3.0,3.0,4.0,5.0
NumberOfTrips,3467.0,3.194693,1.732849,1.0,2.0,3.0,5.0,8.0
Passport,3489.0,0.102608,0.303490,0.0,0.0,0.0,0.0,1.0
PitchSatisfactionScore,3489.0,2.513901,1.115412,1.0,2.0,2.0,3.0,5.0


In [9]:
# Config cell
target = "ProdTaken"
path_in_folder = "./"

cols_category = [
"Age",
"TypeofContact",
"CityTier",
"DurationOfPitch",
"Occupation",
"Gender",
"NumberOfPersonVisiting",
"NumberOfFollowups",
"ProductPitched",
"PreferredPropertyStar",
"NumberOfTrips",
"Passport",
"PitchSatisfactionScore",
"Designation",
"MaritalStatus",
"OwnCar",
"NumberOfChildren",
"Agebin",
"Incomebin"
]

In [10]:
def preprocess(df, replace_dict=None, ce_dict=None):
    # カテゴリカル変数の設定
    df[cols_category] = df[cols_category].fillna(-1)
    if replace_dict is None:
        # countencode, labelencode
        # ce_dict: 列名を入れるとそのカテゴリのデータがどのくらいあるかを返してくれます
        # replace_dict: 列名を入れるとlabelencodeのための数字を返してくれます
        ce_dict = {}
        replace_dict = {}
        for col in cols_category:
            replace_dict[col] = {}
            vc = df[col].value_counts()
            ce_dict[col] = vc
            replace_dict_in_dict = {}
            for i, k in enumerate(vc.keys()):
                replace_dict_in_dict[k] = i
            replace_dict[col] = replace_dict_in_dict
            df[f"{col}_CountEncode"] = df[col].replace(vc).astype(int)
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df, replace_dict, ce_dict

    # test
    else:
        for col in cols_category:
            # カウントエンコード
            test_vals_uniq = df[col].unique()
            ce_dict_in_dict = ce_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in ce_dict_in_dict.keys():
                    ce_dict_in_dict[test_val] = -1
            df[f"{col}_CountEncode"] = df[col].replace(ce_dict_in_dict).astype(int)

            # LabelEncode
            test_vals_uniq = df[col].unique()
            replace_dict_in_dict = replace_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in replace_dict_in_dict.keys():
                    replace_dict_in_dict[test_val] = -1
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df

In [11]:
df_train, replace_dict, ce_dict = preprocess(df_train)
df_test = preprocess(df_test, replace_dict=replace_dict, ce_dict=ce_dict)

In [12]:
df_train.head()


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,PreferredPropertyStar_CountEncode,NumberOfTrips_CountEncode,Passport_CountEncode,PitchSatisfactionScore_CountEncode,Designation_CountEncode,MaritalStatus_CountEncode,OwnCar_CountEncode,NumberOfChildren_CountEncode,Agebin_CountEncode,Incomebin_CountEncode
0,0,2,0,0,2,2,0,2,0,0,...,2147,515,358,739,1150,581,1785,985,843,832
1,1,31,1,1,3,1,0,2,0,2,...,2147,1164,358,739,854,878,1704,985,527,318
2,2,13,0,1,6,2,1,2,1,0,...,2147,163,3131,739,1150,1384,1785,985,100,832
3,3,6,0,0,13,0,1,2,1,2,...,1145,409,3131,103,854,878,1704,985,1126,1003
4,4,25,1,2,10,0,1,2,1,0,...,1145,163,3131,739,1150,646,1704,985,843,832


In [13]:
rows = []
for col in df_train.columns:
    rows.append([col, df_train[col].dtype, df_train[col].isnull().sum(), len(df_train[col].unique())])
pd.DataFrame(rows, columns=["列名", "列の型", "NaNである行の数", "値の種類"])

,列名,列の型,NaNである行の数,値の種類
0,id,int64,0,3489
1,Age,int64,0,45
2,TypeofContact,int64,0,3
3,CityTier,int64,0,3
4,DurationOfPitch,int64,0,34
5,Occupation,int64,0,3
6,Gender,int64,0,2
7,NumberOfPersonVisiting,int64,0,4
8,NumberOfFollowups,int64,0,7
9,ProductPitched,int64,0,5


In [ ]:
#df_train.drop(["id"], axis=1, inplace=True)
#df_test.drop(["id"], axis=1, inplace=True)

In [14]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [15]:
train_x.shape

(3489, 40)

In [16]:
test_x = df_test

In [17]:
test_x.shape

(3489, 41)

In [18]:
test_x = test_x.drop(target, axis=1)

In [ ]:
#df_train.to_csv("/content/drive/MyDrive/Signate/2024summer/dft_train.csv", index=False)
#df_test.to_csv("/content/drive/MyDrive/Signate/2024summer/dft_test.csv", index=False)

# モデル


In [20]:
params_lgb = {
    "n_estimators": 3310,
    "learning_rate": 0.05,
    "colsample_bytree": 0.5,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}

In [21]:
params_cat = {
    "iterations": 2500,  # n_estimatorsに相当
    "learning_rate": 0.01,
    "colsample_bylevel": 0.6,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [22]:
base_models = {
    'LightGBM': lgb.LGBMClassifier(**params_lgb),
    'CatBoost': CatBoostClassifier(**params_cat, early_stopping_rounds=100)
}

In [23]:
def train_base_models(X, y, models, cols_category, n_splits=5):
    meta_features = np.zeros((len(X), len(models)))
    oof_predictions = np.zeros(len(X))

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

    for i, (model_name, model) in enumerate(models.items()):
        print(f"Training {model_name}...")
        oof_predictions_model = np.zeros(len(X))

        for fold, (trn_idx, val_idx) in enumerate(cv.split(X, y), start=1):
            trn_x, trn_y = X.iloc[trn_idx], y.iloc[trn_idx]
            val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]

            if isinstance(model, lgb.LGBMClassifier):
                model.fit(
                    trn_x, trn_y,
                    eval_set=[(val_x, val_y)],
                    callbacks=[lgb.early_stopping(100, verbose=False)],
                    categorical_feature=cols_category,
                )
            elif isinstance(model, CatBoostClassifier):
                train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
                val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)
                model.fit(train_pool, eval_set=val_pool, verbose=False)
            else:
                model.fit(trn_x, trn_y)

            oof_predictions_model[val_idx] = model.predict_proba(val_x)[:, 1]

            auc = roc_auc_score(val_y, oof_predictions_model[val_idx])
            print(f"  Fold {fold}, AUC: {auc:.4f}")

        meta_features[:, i] = oof_predictions_model
        oof_predictions += oof_predictions_model / len(models)

        auc = roc_auc_score(y, oof_predictions_model)
        print(f"{model_name} OOF AUC: {auc:.4f}")

    return meta_features, oof_predictions

In [24]:
def train_meta_model(meta_features, y, meta_model, n_splits=5):
    oof_predictions = np.zeros(len(y))
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)

    for fold, (trn_idx, val_idx) in enumerate(cv.split(meta_features, y), start=1):
        trn_x, trn_y = meta_features[trn_idx], y.iloc[trn_idx]
        val_x, val_y = meta_features[val_idx], y.iloc[val_idx]

        meta_model.fit(trn_x, trn_y)
        oof_predictions[val_idx] = meta_model.predict_proba(val_x)[:, 1]

        auc = roc_auc_score(val_y, oof_predictions[val_idx])
        print(f"Meta Model Fold {fold}, AUC: {auc:.4f}")

    auc = roc_auc_score(y, oof_predictions)
    print(f"Meta Model OOF AUC: {auc:.4f}")

    return oof_predictions

In [25]:
def stacking_predict(X, base_models, meta_model):
    meta_features = np.column_stack([model.predict_proba(X)[:, 1] for model in base_models])
    return meta_model.predict_proba(meta_features)[:, 1]

In [26]:
# ベースモデルの学習とメタ特徴量の生成
meta_features, oof_predictions_base = train_base_models(train_x, train_y, base_models, cols_category)


Training LightGBM...
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001351 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 847
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 40
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
  Fold 1, AUC: 0.8564
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000418 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 847
[LightGBM] [Info] Number of data points in the train set: 2791, number of use

In [27]:
meta_features

array([[0.76435356, 0.7137148 ],
       [0.13405498, 0.14667005],
       [0.20265904, 0.24623595],
       ...,
       [0.08273623, 0.05436369],
       [0.23423968, 0.22185141],
       [0.08678801, 0.1167399 ]])

In [28]:
# メタモデルの定義と学習
from sklearn.linear_model import LogisticRegression
meta_model = LogisticRegression(random_state=0)
oof_predictions_meta = train_meta_model(meta_features, train_y, meta_model)


Meta Model Fold 1, AUC: 0.8657
Meta Model Fold 2, AUC: 0.8279
Meta Model Fold 3, AUC: 0.8068
Meta Model Fold 4, AUC: 0.8591
Meta Model Fold 5, AUC: 0.8306
Meta Model OOF AUC: 0.8346


In [30]:
final_auc = roc_auc_score(train_y, oof_predictions_meta)
print(f"Final Stacking Model OOF AUC: {final_auc:.8f}")

Final Stacking Model OOF AUC: 0.83462809


Final Stacking Model OOF AUC: 0.8343

In [31]:
test_predictions = stacking_predict(test_x, base_models.values(), meta_model)


In [32]:
test_predictions

array([0.06829245, 0.10521269, 0.14005746, ..., 0.81599059, 0.17321355,
       0.04907369])

In [33]:
count = 10

In [34]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/lightcat_{timestamp}_{count:03d}.csv"
ss[1] = test_predictions
ss.to_csv(file_name, header=False, index=False)